importacion de librerias


In [66]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns

In [67]:
df = pd.read_csv('./data/2024_Accidentalidad.csv', delimiter=';')


In [68]:
df.sample(10)

,num_expediente,fecha,hora,localizacion,numero,cod_distrito,distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,tipo_persona,rango_edad,sexo,cod_lesividad,lesividad,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,positiva_droga
22712,2024S019750,14/06/2024,20:30:00,CALL. ALCALA / CALL. MARCELINO ALVAREZ,264,15,CIUDAD LINEAL,Colisión lateral,Despejado,Turismo,Conductor,De 30 a 34 años,Mujer,14.0,Sin asistencia sanitaria,444510.0,4475939.0,N,NaN
36519,2024S028100,07/10/2024,16:40:00,"CALL. O'DONNELL, 64",64,3,RETIRO,Alcance,LLuvia intensa,Camión rígido,Conductor,De 21 a 24 años,Hombre,NaN,NaN,443469.0,4474636.0,N,NaN
19463,2024S017628,24/05/2024,18:40:00,GTA. SANTA MARIA DE LA CABEZA / CALL. EMBAJADORES,1,2,ARGANZUELA,Colisión lateral,Despejado,Motocicleta > 125cc,Conductor,De 45 a 49 años,Hombre,NaN,NaN,440626.0,4472534.0,N,NaN
28939,2024S023497,03/08/2024,8:58:00,CALL. LOPEZ DE HOYOS / CALL. CARTAGENA,86,5,CHAMARTÍN,Alcance,Despejado,Turismo,Pasajero,De 15 a 17 años,Hombre,14.0,Sin asistencia sanitaria,442660.0,4477108.0,N,NaN
17396,2024S015950,10/05/2024,13:10:00,"PASEO. PRADO, 32",32,1,CENTRO,Colisión lateral,Despejado,Furgoneta,Conductor,De 25 a 29 años,Hombre,NaN,NaN,441204.0,4473652.0,N,NaN
17639,2024S016090,11/05/2024,17:30:00,"AVDA. ARCENTALES, 80 (GLORIETA)",80,20,SAN BLAS-CANILLEJAS,Colisión fronto-lateral,Despejado,Turismo,Conductor,De 21 a 24 años,Hombre,14.0,Sin asistencia sanitaria,450889.0,4475315.0,N,NaN
3592,2024S003434,29/01/2024,14:30:00,"CALL. GENERAL RICARDOS, 104",104,11,CARABANCHEL,Colisión lateral,Despejado,Turismo,Conductor,De 25 a 29 años,Mujer,14.0,Sin asistencia sanitaria,438453.0,4471591.0,N,NaN
32897,2024S025964,12/09/2024,14:45:00,"AVDA. MEMORIA, 4",4,9,MONCLOA-ARAVACA,Alcance,Despejado,Turismo,Conductor,De 50 a 54 años,Hombre,14.0,Sin asistencia sanitaria,438442.0,4476761.0,N,NaN
22147,2024S019430,11/06/2024,6:50:00,PASEO. DELICIAS / CALL. MURCIA,10,2,ARGANZUELA,Colisión lateral,Despejado,Turismo,Conductor,Desconocido,Desconocido,NaN,NaN,441153.0,4472941.0,N,NaN
4980,2024S004931,07/02/2024,19:42:00,CALL. HERMANOS GARCIA NOBLEJAS / CALL. MIGUEL ...,43,20,SAN BLAS-CANILLEJAS,Atropello a persona,Despejado,Motocicleta > 125cc,Peatón,De 65 a 69 años,Mujer,7.0,Asistencia sanitaria sólo en el lugar del acci...,446304.0,4475812.0,N,NaN


Analisis del dataset


https://datos.madrid.es/FWProjects/egob/Catalogo/Seguridad/Ficheros/Estructura_ConjuntoDatos_Accidentesv2.pdf


Lo primero que nos damos cuenta cuando leemos el anterior documento es que el año del accidente no viene en la fecha, si no que viene en los 4 primeros char del numero del expediente.


**localizacion** y **numero**: con la localizacion y el numero (numero de la calle) nos parece interesante quedarnosla, pero solo para la parte de entendimiento de los datos del cluster, ya que para el claster es mas util las coordenadas utm


para el codigo del distrito nos va a servir para hacer el cluster, pero como el numero en si no nos dice nada, pues entonces lo que vamos a hacer un mapeo del distrito para entender y tener la correlacion


para el tipo de accidente y para el estado metereologico haremos un mapeo simple de las 13 posibles variables (mapping)


en tipo de vehiculo habia demasiados valroes unicos (33) por lo que hemos intentado generalizar un poco para poder tratar con esta columna, haciendo un mapeo y un posible posterior onehotencoding


para el **rango de edad** vamos


Entendimiento del Dataset


Limpieza


Explicacion de las acciones realizadas


In [69]:
len(df['tipo_vehiculo'].unique())


33

In [70]:
df['tipo_vehiculo'].value_counts()

tipo_vehiculo
Turismo                              27406
Motocicleta hasta 125cc               2668
Furgoneta                             2667
Motocicleta > 125cc                   1469
Autobús                               1264
Camión rígido                          973
Todo terreno                           706
Bicicleta                              670
VMU eléctrico                          665
Ciclomotor                             387
Bicicleta EPAC (pedaleo asistido)      211
Tractocamión                           187
Maquinaria de obras                    151
Otros vehículos con motor              131
Vehículo articulado                     70
Autobús articulado                      69
Sin especificar                         40
Cuadriciclo ligero                      32
Autobus EMT                             17
Ciclo                                   13
Cuadriciclo no ligero                   12
Autocaravana                            11
Otros vehículos sin motor               

In [71]:
map_vehiculos = {
    'Turismo': 'Turismos y vehículos ligeros',
    'Todo terreno': 'Turismos y vehículos ligeros',
    'Autocaravana': 'Turismos y vehículos ligeros',
    'Motocicleta hasta 125cc': 'Motocicletas y ciclomotores',
    'Motocicleta > 125cc': 'Motocicletas y ciclomotores',
    'Ciclomotor': 'Motocicletas y ciclomotores',
    'Bicicleta EPAC (pedaleo asistido)': 'Motocicletas y ciclomotores',
    'Ciclomotor de tres ruedas': 'Motocicletas y ciclomotores',
    'Moto de tres ruedas hasta 125cc': 'Motocicletas y ciclomotores',
    'Moto de tres ruedas > 125cc': 'Motocicletas y ciclomotores',
    'Ciclo de motor L1e-A': 'Motocicletas y ciclomotores',
    'Ciclomotor de dos ruedas L1e-B': 'Motocicletas y ciclomotores',
    'Furgoneta': 'Vehículos de carga',
    'Camión rígido': 'Vehículos de carga',
    'Tractocamión': 'Vehículos de carga',
    'Vehículo articulado': 'Vehículos de carga',
    'Semiremolque': 'Vehículos de carga',
    'Remolque': 'Vehículos de carga',
    'Autobús': 'Vehículos de transporte público',
    'Autobús articulado': 'Vehículos de transporte público',
    'Autobus EMT': 'Vehículos de transporte público',
    'Bicicleta': 'Bicicletas y ciclos no motorizados',
    'Ciclo': 'Bicicletas y ciclos no motorizados',
    'Patinete no eléctrico': 'Bicicletas y ciclos no motorizados',
    'VMU eléctrico': 'Vehículos eléctricos ligeros',
    'Maquinaria de obras': 'Vehículos especializados',
    'Ambulancia SAMUR': 'Vehículos especializados',
    'Otros vehículos con motor': 'Vehículos especializados',
    'Otros vehículos sin motor': 'Vehículos especializados',
    'Sin especificar': 'Sin clasificar/especificar',
    'Cuadriciclo ligero': 'Sin clasificar/especificar',
    'Cuadriciclo no ligero': 'Sin clasificar/especificar'
}

In [72]:
df['tipo_vehiculo'] = df['tipo_vehiculo'].map(map_vehiculos)

Mapeo de rango de edades


In [73]:
map_edad = {
    'Desconocido': -1,
    'Menor de 5 años': 0,
    'De 6 a 9 años': 6,
    'De 10 a 14 años': 10,
    'De 15 a 17 años': 15,
    'De 18 a 20 años': 18,
    'De 21 a 24 años': 21,
    'De 25 a 29 años': 25,
    'De 30 a 34 años': 30,
    'De 35 a 39 años': 35,
    'De 40 a 44 años': 40,
    'De 45 a 49 años': 45,
    'De 50 a 54 años': 50,
    'De 55 a 59 años': 55,
    'De 60 a 64 años': 60,
    'De 65 a 69 años': 65,
    'De 70 a 74 años': 70,
    'Más de 74 años': 74
}

In [74]:
df['rango_edad'] = df['rango_edad'].map(map_edad)

Mapeo de estado meteorológico


In [75]:
df = df.fillna({
    'estado_meteorológico': 'Se desconoce'
})

print(df['estado_meteorológico'].unique())

map_est_met = {
    'Se Desconoce': -1,
    'Despejado': 0,
    'Nublado': 1,
    'Lluvia débil': 2,
    'LLuvia intensa': 3,
    'Granizando': 4,
    'Nevando': 5
}

['Lluvia débil' 'Despejado' 'Se desconoce' 'Nublado' 'LLuvia intensa'
 'Granizando' 'Nevando']


Mapeo Lesividad


In [76]:
print(df['lesividad'].unique())

['Ingreso inferior o igual a 24 horas' 'Sin asistencia sanitaria'
 'Asistencia sanitaria sólo en el lugar del accidente' nan
 'Atención en urgencias sin posterior ingreso'
 'Ingreso superior a 24 horas'
 'Asistencia sanitaria inmediata en centro de salud o mutua'
 'Asistencia sanitaria ambulatoria con posterioridad' 'Fallecido 24 horas']


In [77]:
df = df.fillna({
    'lesividad': 'Sin asistencia sanitaria',
    'cod_lesividad': 0
})

map_lesividad = {
    'Sin asistencia sanitaria': 'NONE',
    'Atención en urgencias sin posterior ingreso': 'LEVE',
    'Ingreso inferior o igual a 24 horas': 'LEVE',
    'Asistencia sanitaria ambulatoria con posterioridad': 'LEVE',
    'Asistencia sanitaria inmediata en centro de salud o mutua': 'LEVE',
    'Asistencia sanitaria sólo en el lugar del accidente': 'LEVE',
    'Ingreso inferior o igual a 24 horas': 'GRAVE',
    'Fallecido 24 horas': 'FALLECIDO'
}

In [78]:
df['lesividad'] = df['lesividad'].map(map_lesividad)

Mapeo Alcohol y drogas


In [79]:
df = df.fillna({
    'positiva_droga': 0
})

map_alcohol = {
    'N': 0,
    'S': 1
}

df['positiva_alcohol'] = df['positiva_alcohol'].map(map_alcohol)

In [ ]:
print('valores de positivos en alcohol: \n {} \n valores de positivos en droga: \n {}'.format(
    df['positiva_alcohol'].value_counts(),df['positiva_droga'].value_counts()
    ))

In [80]:
df.head(10)

,num_expediente,fecha,hora,localizacion,numero,cod_distrito,distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,tipo_persona,rango_edad,sexo,cod_lesividad,lesividad,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,positiva_droga
0,2023S040280,04/01/2024,14:09:00,AVDA. NICETO ALCALA ZAMORA / AUTOV. M-11,3,16,HORTALEZA,Colisión fronto-lateral,Lluvia débil,Motocicletas y ciclomotores,Conductor,55,Hombre,2.0,GRAVE,444913.0,4481427.0,0.0,0.0
1,2023S040280,04/01/2024,14:09:00,AVDA. NICETO ALCALA ZAMORA / AUTOV. M-11,3,16,HORTALEZA,Colisión fronto-lateral,Lluvia débil,Turismos y vehículos ligeros,Conductor,55,Mujer,14.0,NONE,444913.0,4481427.0,0.0,0.0
2,2023S040309,15/02/2024,14:05:00,CALL. TESORO / CALL. MINAS,18,1,CENTRO,Colisión fronto-lateral,Lluvia débil,Bicicletas y ciclos no motorizados,Conductor,25,Hombre,7.0,LEVE,440123.0,4475170.0,0.0,0.0
3,2023S040309,15/02/2024,14:05:00,CALL. TESORO / CALL. MINAS,18,1,CENTRO,Colisión fronto-lateral,Lluvia débil,Motocicletas y ciclomotores,Conductor,35,Hombre,14.0,NONE,440123.0,4475170.0,0.0,0.0
4,2023S040310,18/02/2024,10:40:00,GTA. RUIZ JIMENEZ / CALL. SAN BERNARDO,3,7,CHAMBERÍ,Colisión lateral,Despejado,Turismos y vehículos ligeros,Conductor,25,Hombre,0.0,NONE,440137.0,4475721.0,0.0,0.0
5,2023S040310,18/02/2024,10:40:00,GTA. RUIZ JIMENEZ / CALL. SAN BERNARDO,3,7,CHAMBERÍ,Colisión lateral,Despejado,Turismos y vehículos ligeros,Conductor,55,Hombre,0.0,NONE,440137.0,4475721.0,0.0,0.0
6,2023S040312,25/02/2024,17:20:00,"CALL. PINOS ALTA, 93",93,6,TETUÁN,Colisión lateral,Despejado,Vehículos de carga,Conductor,25,Hombre,14.0,NONE,440709.0,4480102.0,0.0,0.0
7,2023S040312,25/02/2024,17:20:00,"CALL. PINOS ALTA, 93",93,6,TETUÁN,Colisión lateral,Despejado,Vehículos especializados,Conductor,50,Hombre,14.0,NONE,440709.0,4480102.0,0.0,0.0
8,2023S040316,07/03/2024,14:25:00,CTRA. BARRIO DE LA FORTUNA / CALL. PINAR DE SA...,14,10,LATINA,Alcance,Lluvia débil,Vehículos de carga,Conductor,60,Hombre,0.0,NONE,433983.0,4468473.0,0.0,0.0
9,2023S040316,07/03/2024,14:25:00,CTRA. BARRIO DE LA FORTUNA / CALL. PINAR DE SA...,14,10,LATINA,Alcance,Lluvia débil,Turismos y vehículos ligeros,Conductor,25,Hombre,0.0,NONE,433983.0,4468473.0,0.0,0.0


In [81]:
df['rango_edad'].value_counts()

rango_edad
-1     4316
 45    4107
 30    3919
 25    3885
 40    3845
 35    3796
 50    3698
 55    2869
 21    2627
 60    1990
 18    1268
 74     963
 65     956
 70     576
 15     374
 0      367
 10     349
 6      260
Name: count, dtype: int64

In [82]:
df.columns

Index(['num_expediente', 'fecha', 'hora', 'localizacion', 'numero',
       'cod_distrito', 'distrito', 'tipo_accidente', 'estado_meteorológico',
       'tipo_vehiculo', 'tipo_persona', 'rango_edad', 'sexo', 'cod_lesividad',
       'lesividad', 'coordenada_x_utm', 'coordenada_y_utm', 'positiva_alcohol',
       'positiva_droga'],
      dtype='object')

In [83]:
df['cod_lesividad'].unique()

array([ 2., 14.,  7.,  0.,  1.,  3.,  6.,  5.,  4.])

In [84]:
df['cod_lesividad'] = df['cod_lesividad'].astype(int)

In [85]:
df['year'] = df['num_expediente'].astype(str).str[:4].astype(int)
df['id'] = df['num_expediente'].astype(str).str[-6:]

In [86]:
df['year'].value_counts()

year
2024    40154
2023       11
Name: count, dtype: int64

In [87]:
df = pd.get_dummies(df, columns=['tipo_persona'])

In [ ]:
df.head(10)

,cod_distrito,rango_edad,cod_lesividad,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,positiva_droga,year
count,40165.000000,40165.000000,40165.000000,40155.000000,4.015500e+04,40027.000000,40165.000000,40165.000000
mean,9.792108,35.013619,5.730138,442057.893612,4.474652e+06,0.028706,0.004233,2023.999726
std,5.640908,19.168981,6.190382,3491.643374,3.985226e+03,0.166980,0.064921,0.016547
min,1.000000,-1.000000,0.000000,429177.000000,4.463890e+06,0.000000,0.000000,2023.000000
25%,5.000000,25.000000,0.000000,439872.500000,4.471562e+06,0.000000,0.000000,2024.000000
50%,10.000000,35.000000,2.000000,441722.000000,4.474663e+06,0.000000,0.000000,2024.000000
75%,14.000000,50.000000,14.000000,443992.000000,4.477471e+06,0.000000,0.000000,2024.000000
max,21.000000,74.000000,14.000000,454754.000000,4.491676e+06,1.000000,1.000000,2024.000000
